In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from keras.utils import np_utils

In [6]:
image_dir = 'kannada_1/img'

In [7]:
preds = {
    0 : 'ಅ', 
    1 : 'ಆ',
    2 : 'ಇ',
    3 : 'ಈ',
    4 : 'ಉ',
    5 : 'ಊ',
    6 : 'ಋ',
    7 : 'ೠ',
    8 : 'ಎ',
    9 : 'ಏ',
    10 : 'ಐ',
    11 : 'ಒ',
    12 : 'ಓ',
    13 : 'ಔ',
    14 : 'ಅಂ',
    15 : 'ಅಃ',
    16 : 'ಕ',
    17 : 'ಖ',
    18 : 'ಗ',
    19 : 'ಘ',
    20 : 'ಙ',
    21 : 'ಚ',
    22 : 'ಛ',
    23 : 'ಜ',
    24 : 'ಝ', 
    25 : 'ಞ',
    26 : 'ಟ',
    27 : 'ಠ',
    28 : 'ಡ',
    29 : 'ಢ',
    30 : 'ಣ',
    31 : 'ತ',
    32 : 'ಥ',
    33 : 'ದ',
    34 : 'ಧ',
    35 : 'ನ',
    36 : 'ಪ',
    37 : 'ಫ',
    38 : 'ಬ',
    39 : 'ಭ',
    40 : 'ಮ',
    41 : 'ಯ',
    42 : 'ರ',
    43 : 'ಱ',
    44 : 'ಲ',
    47 : 'ವ',
    48 : 'ಶ',
    49 : 'ಷ',
    50 : 'ಸ',
    51 : 'ಹ',
    45 : 'ಳ',
    46 : 'ೞ',
    52 : '೦',
    53 : '೧',
    54 : '೨',
    55 : '೩',
    56 : '೪',
    57 : '೫',
    58 : '೬',
    59 : '೭',
    60 : '೮',
    61 : '೯'
}

In [8]:
images = []
for g_id in os.listdir(image_dir):
    for idx, img_path in enumerate(os.listdir(os.path.join(image_dir, g_id))):
        imgs = cv2.imread(image_dir + "/" + g_id + "/" + str(idx + 1) + ".png", 0)
        imgs = cv2.resize(imgs, (64, 64))
        images.append((np.array(imgs), int(g_id)))   

NotADirectoryError: [Errno 20] Not a directory: 'kannada_1/img/Kannada_Character_Recognition.h5'

In [ ]:
images = shuffle(shuffle(images))

In [ ]:
def split_images_labels(imgs):
    images = []
    labels = []
    for img, label in imgs:
        images.append(img)
        labels.append(label)
    return images, labels

In [ ]:
imgs, labels = split_images_labels(images)

In [ ]:
imgs = np.array(imgs)

In [ ]:
imgs.shape

In [ ]:
def plot_digit(img):
    plt.figure(figsize = (2, 2))
    plt.imshow(img, cmap = 'gray')
    plt.show()

In [ ]:
def show_letter(img_index):
    print("Actual Base Letter : ", preds[labels[img_index]])
    plot_digit(imgs[img_index])

In [ ]:
show_letter(1001)

In [ ]:
def split_data(images, labels):
    
    print("Length of images_labels", len(labels))

    train_images = images[:int(5/6*len(images))]
    print("Length of train_images", len(train_images))
    

    train_labels = labels[:int(5/6*len(labels))]
    print("Length of train_labels", len(train_labels))
    

    test_images = images[int(5/6*len(images)):]
    print("Length of test_images", len(test_images))
    

    test_labels = labels[int(5/6*len(labels)):]
    print("Length of test_labels", len(test_labels))
    
    return train_images, train_labels, test_images,test_labels

In [ ]:
train_images, train_labels, test_images, test_labels = split_data(imgs, labels)

In [ ]:
train_images.shape, test_images.shape

In [ ]:
train_images = train_images.reshape(-1, 64, 64, 1)
test_images = test_images.reshape(-1, 64, 64, 1)

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
#train_labels = np_utils.to_categorical(train_labels)
#test_labels = np_utils.to_categorical(test_labels)

In [ ]:
data_augmentation = tf.keras.Sequential([
  keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters = 10, kernel_size = (7, 7), padding = 'SAME', activation = 'relu', input_shape = (64, 64, 1)),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(filters = 64, kernel_size = (5, 5), activation = 'relu'),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(filters = 128, kernel_size = (5, 5), activation = 'relu'),
    keras.layers.Conv2D(filters = 128, kernel_size = (5, 5), activation = 'relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(84, activation = 'relu'),
    keras.layers.Dense(62, activation = 'softmax')
    ])
    
#Training Process

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(train_images, train_labels, epochs = 20, 
             validation_data = (test_images, test_labels))

In [ ]:
model.save(os.path.join(image_dir, 'Kannada_Character_Recognition.h5'))

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
def predict_character(img):
    img_pred = model.predict(img.reshape(-1, 64, 64, 1))
    plot_digit(img)
    print('Predicted Character : ', preds[np.argmax(img_pred)])

In [ ]:
import random

In [ ]:
predict_character(test_images[1260])